In [120]:
import requests
import csv
import time
from bs4 import BeautifulSoup
from collections import Counter
import praw
import pandas as pd
import datetime as dt #only if you want to analyze the date created feature
import json 
from types import SimpleNamespace
import sys
import datetime as dt
import importlib

## Code for Constructing the Dataset for our CS 224U Project

### PRAW
1. Generated a prelim list of users who've posts on r/Anxiety using ParseHub
2. Using the Reddit API through praw (initially was trying to use BeautifulSoup, but Reddit API is easier) to fetch posts by users
3. Future work: Optimise python processing 


### Four categories
1. ANXIOUS-USERS Mental Illness Related Subreddits - ('men')
2. ANXIOUS-USERS Anxiety Related Subreddits (subset of first group) - ('anx')
3. ANXIOUS-USERS Neutral Subreddits (Mutually exclusive from first group) - ('unr')

### Utils

In [33]:
class SetEncoder(json.JSONEncoder):
    '''Using SetEncoder to JSONify our individual result sets'''
    def default(self, obj):
        if isinstance(obj, set): return list(obj)
        return json.JSONEncoder.default(self, obj)
    
reddit = praw.Reddit(client_id='A1ufXY3vdi2vpA', 
                 client_secret='nO1tFwJCFGiegR4NQzcMEbPze0g', 
                 user_agent='Script', 
                 username='sophiemiya', 
                 password='stevie13weedR')

from psaw import PushshiftAPI
api = PushshiftAPI()

### Collect User List

In [21]:
import ast
with open('dataset/all-users-dont-change.txt','r') as f:
    my_set = str(ast.literal_eval(f.read()))
    unique = ast.literal_eval(my_set)

In [ ]:
# COLLECT USERS THROUGH PRAW (LATEST 1000 POSTS EACH SUB)

# unique = set()

# def add_to_unique_users(sub):
#     print("Get from praw for sub: ", sub)
#     panic_party_posts = reddit.subreddit(sub).top(limit=1000)
#     panic_users = [post.author.name for post in panic_party_posts if post.author is not None]
#     unique.update(panic_users)

# for sub in SUBS_ANX:
#     add_to_unique_users(sub)

In [ ]:
# COLLECT USERS THROUGH PSAW (1000 POSTS EACH SUB BY YEAR)
# def get_from_psaw(start_epoch, sub):

#     return [sub.author for sub in api.search_submissions(before=start_epoch,
#                             subreddit=sub,
#                             filter=['author'],
#                             limit=1000)]
    
# for sub in SUBS_ANX:
#     print("Get from psaw for sub: ", sub)
#     unique.update(get_from_psaw(int(dt.datetime(2019, 12, 1).timestamp()), sub))
#     unique.update(get_from_psaw(int(dt.datetime(2018, 12, 1).timestamp()), sub))
#     unique.update(get_from_psaw(int(dt.datetime(2017, 12, 1).timestamp()), sub))

# dump_data('dataset/all-users-dont-change.txt', unique)

In [121]:
# TOTAL NUMBER OF UNIQUE USERS
len(unique)

14299

In [152]:
LIM = 30 # num of posts per user
categories = get_fresh_counters()

In [153]:
results = {} # stores actual result here
results_user_names = [] # stores final user names here


# COUNTERS

result_user_count = 0 # counts the total valid users we get by the end
result_post_count = 0 # counts the total valid posts we get by the end

In [154]:
import tryutils
importlib.reload(tryutils)
from tryutils import SUBS_ANX, SUBS_MEN, get_fresh_counters, get_post_type,\
dump_data, process_user, init_vars, get_dict_from_submission, process_for_category

def get_data_for_users(start, end):
    global categories, result_user_count, result_post_count

    user_count = 0 

    # get the submissions
    users_to_process = list(unique)[start:end]

    users_submissions = [(reddit.redditor(user).submissions.new(limit=LIM), user) for user in users_to_process]
    user_submissions_2 = [] # need this because of Praw iterator related bug

    # main processing happens now:

    for user_submissions, user in users_submissions: # for each user
        user_count = process_user(user, user_count, (end-start)) # print 

        try: # 404 HTTP errors
            user_submissions = list(user_submissions)

            if len(user_submissions) < 4: continue # user should have at least 4 posts

            user_result, count_related, count_unrelated  = init_vars() 

            submissions_2 = [] # need this because of Praw iterator related bug
                        
            for submission in user_submissions: # quick pass to see if this user's posts are relevant
                if count_related >= 2 and count_unrelated >= 2: break # is relevant, stop here

                if not submission.title or not submission.selftext or submission.selftext == "[removed]": continue 

                if submission.subreddit.display_name in SUBS_ANX: count_related += 1
                elif submission.subreddit.display_name not in SUBS_MEN: count_unrelated += 1
                    
                submissions_2.append(submission)
                    
            if(count_related >= 2 and count_unrelated >= 2): 
                anx_count = mental_count = neutral_count = 0
                try:
                    for submission in submissions_2:
                        tag = ""
                        
                        if submission.subreddit.display_name in SUBS_ANX: 
                            anx_count += 1
                            tag = "anx"
                        elif submission.subreddit.display_name in SUBS_MEN:
                            mental_count += 1
                            tag = "men"
                        else: 
                            neutral_count += 1 
                            tag = "neu"
                            
                        if (anx_count > 2 and tag =="anx") or \
                        (neutral_count > 2 and tag == "neu") or \
                        (mental_count > 2 and tag == "men"): continue            
 
                        post = get_dict_from_submission(submission, user)
    
                        categories = process_for_category(post, submission.subreddit.display_name, categories)
                        user_result.append(post)

                except Exception: continue
                results_user_names.append(user) # add user to the list of final users used
                results[user] = (user_result) # add result to final result
                result_user_count += 1 # increment the user count
                result_post_count += len(user_result)


        except Exception:
            continue

Batch: 0 

496/500 users

Batch: 1 

496/500 users

Batch: 2 



KeyboardInterrupt: 

In [ ]:
batch_count = 22
for i in range(11500,len(unique)):
    print("Batch:", batch_count)
    get_data_for_users(i, i+500)
    print("\n")
    i += 500
    batch_count += 1

Batch: 22
496/500 users

Batch: 23


In [164]:
print("Result user count: ", result_user_count)
print("\nResult post count: ", result_post_count)

print("\nCounter of mental posts: ", categories.men.counter.most_common(10))
print("Counter of anx posts: ", categories.anx.counter.most_common(10))
print("Counter of unrelated posts: ", categories.unr.counter.most_common(10))

print("\nResult of mental posts", len(categories.men.result_list))
print("Result of anx posts", len(categories.anx.result_list))
print("Result of unrelated posts", len(categories.unr.result_list))

Result user count:  1795

Result post count:  8254

Counter of mental posts:  [('depression', 287), ('AskDocs', 139), ('offmychest', 92), ('OCD', 92), ('SuicideWatch', 58), ('antidepressants', 50), ('raisedbynarcissists', 49), ('bipolar', 45), ('TrueOffMyChest', 37), ('CPTSD', 30)]
Counter of anx posts:  [('Anxiety', 1427), ('HealthAnxiety', 584), ('socialanxiety', 534), ('AnxietyDepression', 342), ('zoloft', 261), ('PanicAttack', 181), ('Anxietyhelp', 177), ('panicdisorder', 66), ('PanicParty', 18)]
Counter of unrelated posts:  [('lexapro', 39), ('NoFap', 34), ('Advice', 33), ('personalfinance', 32), ('relationship_advice', 31), ('lonely', 30), ('relationships', 27), ('socialskills', 26), ('Dentistry', 25), ('migraine', 25)]

Result of mental posts 1074
Result of anx posts 3590
Result of unrelated posts 3590


In [ ]:
dump_data('dataset-v1/reddit-data.json', results)

dump_data('dataset-v1/reddit-posts-mental.json', categories.men.result_list)
dump_data('dataset-v1/reddit-posts-anxiety.json', categories.anx.result_list)
dump_data('dataset-v1/reddit-posts-unrelated.json', categories.unr.result_list)

dump_data('dataset-v1/reddit-counter-unrelated.txt', categories.unr.counter.most_common())
dump_data('dataset-v1/reddit-counter-mental.txt', categories.men.counter.most_common())
dump_data('dataset-v1/reddit-counter-anxiety.txt', categories.anx.counter.most_common())

dump_data('dataset-v1/stats.txt', "Result of mental posts: {} \
        Result of anx posts: {} \
        Result of unrelated posts: {}"\
         .format(len(categories.men.result_list),\
                 len(categories.anx.result_list),\
                 len(categories.unr.result_list))